In [1]:
import igraph as ig
import pandas as pd
import numpy as np
import seaborn as sns
from compress_pickle import load, dump
from statistics import mean, median
from c_means_clustering import network_c_means
from c_means_clustering_alt import network_c_means_alt
from copy import deepcopy
import itertools as it
import matplotlib.pyplot as plt
from math import log10
from ease import EASE

plt.style.use('bmh')

path = './files/'

def open_pickle(file):
    with open(path+file, 'rb') as pickle_file:
        return load(path=pickle_file, compression='infer')

regulome_network_edges = pd.read_csv(filepath_or_buffer=path+'/human_regulome_pd.gz', compression='infer')
regulome_network_edges = regulome_network_edges.astype({'ProteinAid': 'str', 'ProteinBid':'str'})

regulome_graph = ig.Graph.DataFrame(regulome_network_edges, directed=False, use_vids=False)
proteins = pd.concat([regulome_network_edges['ProteinAid'], regulome_network_edges['ProteinBid']]).unique()

clusters_dict = open_pickle("c_means_leiden_clusters.gz")
msigdb_c3_tft_dict = open_pickle('msigdb_c3_tft_dict.pkl')

In [ ]:
# def leiden_clustering(graph, res, n_iter, beta=0.05):
#     return graph.community_leiden(objective_function='modularity',
#                             weights='PPV',
#                             resolution=res,
#                             beta=beta,
#                             n_iterations=n_iter)

# leiden_clusters = leiden_clustering(regulome_graph, 8, 20)
# clusters_dict = {n : set(regulome_graph.vs[node]['name'] for node in cluster) for (n, cluster) in enumerate(leiden_clusters) if len(cluster) > 5 and len(cluster) < 200}

# with open('./files/c_means_leiden_clusters.gz', 'wb') as file:
#     dump(clusters_dict, file, compression='infer')


In [2]:
path_len_dict = open_pickle("path_lengths_dict.gz")

In [3]:
for distance_metric in ['edge_ratio', 'path_len']:
    for perc in [0.98, 0.99, 0.999]:
        if distance_metric == 'path_len':
            path_len = path_len_dict
        else:
            pat_len = None
        results = network_c_means(regulome_graph, deepcopy(clusters_dict), 2, n_iter=5, optimize=True, percentile=perc, distance_measure=distance_metric, path_lengths=None, cores=8)

        with open(path+f"c_means_5iter_{distance_metric}_{int(perc*1000)}.gz") as file:
            dump(results, file, compresion='infer')

<function edge_ratio at 0x7f00e4bc0e00>
Iteration 1 of 5


100%|██████████| 15041/15041 [00:05<00:00, 2632.70it/s]


Updating Clusters: percentile 0.98
Iteration 2 of 5


100%|██████████| 15041/15041 [00:37<00:00, 399.35it/s] 


Updating Clusters: percentile 0.98
Iteration 3 of 5


  0%|          | 0/15041 [00:00<?, ?it/s]

: 